## 미니 프로젝트!
---
* API를 통해서 라이엇 데이터 수집하기(로우 데이터 DF 만들기) - 티어는 자유
* 불러온 데이터를 전처리하기(중복 제거, N/A값 제거) matches, timeline 합쳐서 최소 컬럼 25개로.
* 위의 과정을 함수화해서 모듈 만들기
---
* 전처리된 로우 데이터를 오라클 or mySql에 저장하기 -> 최소 5만 레코드
* 팀과 함께 상의해서 흥미롭고 재밌을 것 같은 주제 3가지 선정
* 지표 분석을 통해 나온 DF를 시각화 모듈을 사용해서 표현하기(3개 중 2개 필수)
---
* 전처리부터 시각화까지 캡쳐해서 폴더에 저장
* 개발 보고서, ppt를 만들어서 간단하게 발표 예정
* 모든 .py 파일, .ipynb 파일, 사진, 개발보고서, ppt 다 하나의 폴더로 압축
* (강사분 이메일 주소)
---
목요일 점심 전까지 준비 후 오후에 발표!

### 주제
---
* 미아핑의 횟수와 게임의 승패 여부 조합
* 에픽 몬스터 스틸에 따른 평균 승률의 변동 여부
* 라인마다 시간대 별로 골드를 가장 많이 획득한 챔피언

### import 부

In [1]:
import my_utils as mu
import pandas as pd
import numpy as np
import plotly
import plotly.express as px

In [2]:
# 데이터베이스를 데이터 프레임으로 구조화 (코드 작성 기준 98872 레코드)
df = mu.oracle_totalExecute('select * from lol_mp')

oracle open!


d:\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


oracle close!


### (box) 미아핑과 승패 여부

In [3]:
# 'MATCHID'를 기준으로 이긴 팀과 진 팀의 미아 핑 횟수를 구함
win_team = df[df['WIN'] == 'True'][['MATCHID','ENEMYMISSINGPINGS']]\
    .groupby('MATCHID').sum().rename({'ENEMYMISSINGPINGS': 'WIN_TEAM'}, axis=1)
lose_team = df[df['WIN'] == 'False'][['MATCHID','ENEMYMISSINGPINGS']]\
    .groupby('MATCHID').sum().rename({'ENEMYMISSINGPINGS': 'LOSE_TEAM'}, axis=1)

In [4]:
# 구한 데이터를 그래프화 가능한 형태로 데이터 프레임을 재구성
mia_elem = [['Win Team', v] for v in win_team['WIN_TEAM']] + [['Lose Team', v] for v in lose_team['LOSE_TEAM']]
mia_df = pd.DataFrame(mia_elem, columns=['is_Win', 'MIA Ping Count'])

In [5]:
# Box 그래프로 출력
fig = px.box(
    data_frame=mia_df,
    x='MIA Ping Count',
    y='is_Win'
)
plotly.offline.iplot(fig)

### (bar) 에픽 몬스터 스틸과 승률

In [6]:
# 사용할 데이터 추출 및 정제
steal_data = df[['MATCHID','CHAMPIONNAME','TEAMID','EPICMONSTERSTEALS','WIN']]\
    .sort_values(['MATCHID','TEAMID'], ignore_index=True)\
    .replace(100, 'Red').replace(200, 'Blue')

In [7]:
# 에픽 몬스터 스틸이 있었던 게임을 추출 및 정제
steal_is = steal_data[['MATCHID','EPICMONSTERSTEALS']]\
    .groupby('MATCHID').sum().rename({'EPICMONSTERSTEALS': 'IS_STEAL'}, axis=1)
steal_is = steal_is[steal_is['IS_STEAL'] > 0].applymap(lambda v: True)

In [8]:
# 데이터에서 에픽 몬스터 스틸이 있는 게임만 남긴 뒤 'MATCHID'와 'TEAMID'로 그룹화,
# 'STEAL' 컬럼으로 스틸 수가 1회 이상인 팀을 표시
steal_merge = pd.merge(steal_data, steal_is, on='MATCHID', how='left')\
    .dropna().groupby(['MATCHID','TEAMID','WIN']).sum()\
    .reset_index().rename({'EPICMONSTERSTEALS': 'STEAL'}, axis=1)
steal_merge['STEAL'] = steal_merge['STEAL'].apply(lambda v: 1 if v > 0 else 0)

In [9]:
# 'STEAL' 컬럼을 이용해 쌍방 스틸이 있는 게임을 찾아 제거
isolate = steal_merge.groupby('MATCHID').sum().applymap(lambda v: np.nan if v!=1 else 1)\
    .rename({'STEAL': 'SINGLE_STEAL'}, axis=1).reset_index()
steal_merge = pd.merge(steal_merge, isolate, on='MATCHID').dropna().drop('SINGLE_STEAL', axis=1)

In [10]:
# 스틸 한 팀과 당한 팀을 나눠 각자의 승률을 구해 그래프에 맞는 데이터 프레임으로 재구성
stole_true = steal_merge[steal_merge['STEAL']==1]
stealer = round(len(stole_true[stole_true['WIN']=='True']) / len(stole_true) * 100, 2)
stole_false = steal_merge[steal_merge['STEAL']==0]
stealed = round(len(stole_false[stole_false['WIN']=='True']) / len(stole_false) * 100, 2)
steal_epic_df = pd.DataFrame([['스틸한 팀', stealer], ['스틸 당한 팀', stealed]], columns=['Steal', 'WinRate'])

In [11]:
# bar 그래프 작성
fig = px.bar(
    data_frame=steal_epic_df,
    x='Steal',
    y='WinRate',
    color='Steal',
    text='WinRate',
    width=500
)
fig.update_yaxes(range=[0, 100])
fig.update_traces(textposition='outside')
plotly.offline.iplot(fig)

### (grouped bar) 라인마다 시간대 별 최다 골드 획득 챔피언

In [12]:
# 각 챔피언의 라인 별 등장 횟수가 30회 이상인 챔피언, 라인만 추출
gold_cnt = df[['CHAMPIONNAME','TEAMPOSITION', 'PARTICIPANTID']]\
    .groupby(['CHAMPIONNAME', 'TEAMPOSITION']).count().rename({'PARTICIPANTID': 'PICK_COUNT'}, axis=1)
gold_cnt = gold_cnt[gold_cnt['PICK_COUNT']>=30]\
    .sort_values(['CHAMPIONNAME', 'PICK_COUNT'], ascending=[True, False])

In [13]:
# 각 챔피언의 라인 별 2분당 골드량의 평균을 저장
gold_avg = df[['CHAMPIONNAME','TEAMPOSITION'] + (gcol := [v for v in df.columns[-11:]])]\
    .replace(0, np.nan).groupby(['CHAMPIONNAME','TEAMPOSITION']).mean()\
    .applymap(lambda v: round(v, 1), na_action='ignore')

In [14]:
# 추출한 챔피언과 라인 데이터에 분당 골드량 데이터를 병합
gold_df = pd.merge(gold_cnt, gold_avg, on=['CHAMPIONNAME','TEAMPOSITION']).applymap(lambda v: round(v))
gold_df.reset_index(inplace=True)

In [15]:
# 2분당 라인별 최고 골드 획득 챔피언을 검색해 그래프화 가능한 형태로 데이터 프레임 재구성
gold_raw_data = []
for lane in ['TOP','JUNGLE','MIDDLE','BOTTOM','UTILITY']:
    for time in (curr_lane := gold_df[gold_df['TEAMPOSITION']==lane].set_index('CHAMPIONNAME')[gcol]).columns:
        value = curr_lane[[time]].sort_values(time).iloc[-1]
        gold_raw_data.append([lane, time, value.name, value[time]])
max_gold_df = pd.DataFrame(gold_raw_data, columns=['LANE','TIME','CHAMPIONNAME','GOLD'])

In [16]:
# 그룹화된 bar 그래프 작성
fig = px.bar(
    data_frame=max_gold_df,
    x='TIME',
    y='GOLD',
    text='CHAMPIONNAME',
    color='LANE',
    barmode='group'
)
plotly.offline.iplot(fig)